In [31]:
# Install necessary libraries
# !pip install ultralytics -q

In [32]:
import os
import cv2
import numpy as np

# Define paths
base_path = "images"  # Your dataset folder
true_path = os.path.join(base_path, "true")
false_path = os.path.join(base_path, "false")
output_path = "dataset"
labels_path = os.path.join(output_path, "labels")
images_path = os.path.join(output_path, "images")

# Create YOLO folder structure
folders = [
    os.path.join(images_path, "train"),
    os.path.join(images_path, "val"),
    os.path.join(images_path, "test"),
    os.path.join(labels_path, "train"),
    os.path.join(labels_path, "val"),
    os.path.join(labels_path, "test"),
]
for folder in folders:
    os.makedirs(folder, exist_ok=True)

# Function to split data into train/val/test
import random
def split_and_copy(src_folder, dst_folder, split_ratios=(0.7, 0.2, 0.1)):
    images = [f for f in os.listdir(src_folder) if f.endswith((".jpg", ".png", ".jpeg"))]
    random.shuffle(images)
    n_train = int(len(images) * split_ratios[0])
    n_val = int(len(images) * split_ratios[1])

    train, val, test = images[:n_train], images[n_train:n_train + n_val], images[n_train + n_val:]

    for split, split_name in zip([train, val, test], ["train", "val", "test"]):
        for image in split:
            src = os.path.join(src_folder, image)
            dst = os.path.join(dst_folder, split_name, image)
            os.makedirs(os.path.dirname(dst), exist_ok=True)
            cv2.imwrite(dst, cv2.imread(src))
    return train, val, test

# Split "true" and "false" images into train/val/test
true_train, true_val, true_test = split_and_copy(true_path, images_path)
false_train, false_val, false_test = split_and_copy(false_path, images_path)

# Generate bounding box labels for "true" images
def generate_true_labels(image_list, image_folder, output_folder, class_id=0):
    for image_name in image_list:
        image_path = os.path.join(image_folder, image_name)
        label_file = os.path.splitext(image_name)[0] + ".txt"
        label_path = os.path.join(output_folder, label_file)

        image = cv2.imread(image_path)
        height, width, _ = image.shape

        # Default bounding box (centered, covers a large portion of the image)
        x_center = 0.5
        y_center = 0.5
        box_width = 0.5
        box_height = 0.5

        # Save bounding box
        with open(label_path, "w") as f:
            f.write(f"{class_id} {x_center} {y_center} {box_width} {box_height}\n")

# Generate empty labels for "false" images
def generate_false_labels(image_list, output_folder):
    for image_name in image_list:
        label_file = os.path.splitext(image_name)[0] + ".txt"
        label_path = os.path.join(output_folder, label_file)
        open(label_path, "w").close()  # Create an empty file

# Generate labels for train/val/test splits
generate_true_labels(true_train, os.path.join(images_path, "train"), os.path.join(labels_path, "train"))
generate_true_labels(true_val, os.path.join(images_path, "val"), os.path.join(labels_path, "val"))
generate_true_labels(true_test, os.path.join(images_path, "test"), os.path.join(labels_path, "test"))

generate_false_labels(false_train, os.path.join(labels_path, "train"))
generate_false_labels(false_val, os.path.join(labels_path, "val"))
generate_false_labels(false_test, os.path.join(labels_path, "test"))

print("Dataset processing complete!")

Dataset processing complete!


In [33]:
# Create the dataset YAML file
data_yaml = f"""
train: {os.path.abspath(os.path.join(images_path, 'train'))}
val: {os.path.abspath(os.path.join(images_path, 'val'))}

nc: 1
names: ['mine']
"""

with open("data.yaml", "w") as f:
    f.write(data_yaml)

print("data.yaml created!")

data.yaml created!


In [34]:
from ultralytics import YOLO

# Train YOLOv8 model
model = YOLO("yolov8n.pt")  # Use YOLOv8 Nano for speed
model.train(
    data="data.yaml",  # Dataset configuration
    epochs=10,         # Number of training epochs
    imgsz=416,         # Image size
    project="results", # Specify custom logging folder (e.g., "results" in your project folder)
    name="train_run",  # Name of the training run
    exist_ok=True      # Overwrite existing folder if it exists
)

print("Model training complete!")

New https://pypi.org/project/ultralytics/8.3.39 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.38  Python-3.11.10 torch-2.5.1+cpu CPU (Intel Core(TM) i7-10510U 1.80GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=None, workers=8, project=results, name=train_run, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fals

train: Scanning C:\Users\alesa\Desktop\Magistrale\ESILV\Veille et enjeux\mine-recognition\dataset\labels\train... 223 images, 75 backgrounds, 0 corrupt: 100%|██████████| 227/227 [00:0


train: New cache created: C:\Users\alesa\Desktop\Magistrale\ESILV\Veille et enjeux\mine-recognition\dataset\labels\train.cache


val: Scanning C:\Users\alesa\Desktop\Magistrale\ESILV\Veille et enjeux\mine-recognition\dataset\labels\val... 88 images, 27 backgrounds, 0 corrupt: 100%|██████████| 88/88 [00:01<00:00

val: New cache created: C:\Users\alesa\Desktop\Magistrale\ESILV\Veille et enjeux\mine-recognition\dataset\labels\val.cache


...

      10/10         0G     0.6134      0.857     0.9165          2        416: 100%|██████████| 15/15 [00:56<00:00,  3.77s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:14<00:00,  4.76s/it]

                   all         88         61      0.998          1      0.995      0.814



10 epochs completed in 0.177 hours.
Optimizer stripped from results\train_run\weights\last.pt, 6.2MB
Optimizer stripped from results\train_run\weights\best.pt, 6.2MB

Validating results\train_run\weights\best.pt...
Ultralytics 8.3.38  Python-3.11.10 torch-2.5.1+cpu CPU (Intel Core(TM) i7-10510U 1.80GHz)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:11<00:00,  3.92s/it]


                   all         88         61      0.999          1      0.995      0.834
Speed: 1.5ms preprocess, 43.0ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to results\train_run
Model training complete!


In [41]:
# Path to test images
test_images_path = os.path.join(images_path, "test")
output_folder = "annotated_results"
os.makedirs(output_folder, exist_ok=True)

# Load trained model
trained_model_path = "C:\\Users\\alesa\Desktop\Magistrale\ESILV\Veille et enjeux\mine-recognition\\results\\train_run\weights\\best.pt"  # Update this if necessary
model = YOLO(trained_model_path)

# Run inference on test images
test_images = [f for f in os.listdir(test_images_path) if f.endswith((".jpg", ".png", ".jpeg"))]

for image_name in test_images:
    image_path = os.path.join(test_images_path, image_name)
    results = model(image_path)

    # Save annotated image
    annotated_image = results[0].plot()
    output_path = os.path.join(output_folder, f"annotated_{image_name}")
    cv2.imwrite(output_path, annotated_image)

print(f"Annotated images saved to: {output_folder}")

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:813: error: (-215:Assertion failed) !buf.empty() in function 'cv::imdecode_'


In [36]:
from ultralytics import YOLO

# Load the trained model
trained_model_path = "results/train_run/weights/best.pt"  # Update the path to your best model
model = YOLO(trained_model_path)

# Custom results directory
results_dir = "results"  # Directory where validation results will be saved

# Evaluate the model and save results to custom directory
results = model.val(project=results_dir, name="val_run", exist_ok=True)

# Print key metrics
print("Model Performance:")
print(f"mAP@0.5: {results.box.map50:.4f}")   # Mean Average Precision at IoU=0.5
print(f"mAP@0.5:0.95: {results.box.map:.4f}")  # Mean Average Precision at IoU=0.5:0.95

# Results saved location
print(f"Results saved to: {results_dir}/val_run")


Ultralytics 8.3.38  Python-3.11.10 torch-2.5.1+cpu CPU (Intel Core(TM) i7-10510U 1.80GHz)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\alesa\Desktop\Magistrale\ESILV\Veille et enjeux\mine-recognition\dataset\labels\val.cache... 88 images, 27 backgrounds, 0 corrupt: 100%|██████████| 88/88 [00:00
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:10<00:00,  1.80s/it]


                   all         88         61      0.999          1      0.995      0.832
Speed: 0.6ms preprocess, 37.5ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to results\val_run
Model Performance:
mAP@0.5: 0.9950
mAP@0.5:0.95: 0.8318
Results saved to: results/val_run
